
TO-KNOW :
    -ce sera x (aleatoire) lettres choisies (aleatoirement) 
    -dexuieme fonctionnalité sera que on peut choisir son captcha 

TO-DO :
    -choisir la police 
    voir : https://www.dafont.com/ordionally.font,https://www.dafont.com/wasiat.font,https://www.dafont.com/bongkerspop.font, https://www.dafont.com/love-in-the-sky.font, https://www.dafont.com/nighty-tales-gt.font, https://www.dafont.com/witchi.font
    chercher une qui fais vrmt "fait main" 
    
    
    -voir si je veux les miniscules et les majuscules (ou un des deux, ou un mix des deux->en fct du dataset)

In [12]:
from PIL import Image,ImageDraw,ImageFont
from random import choice, randint
from math import sin, ceil

In [13]:
def create_sequence_random():
    lettres = ["a","b","c","d","e","f","g","h","i","j","k","l","m","n","o","p","q","r","s","t","u","v","w","x","y","z","0","1","2","3","4","5","6","7","8","9"]
    nb_lettres = randint(8,9)
    sequence = ""
    for x in range(nb_lettres):
        lettre = choice(lettres)  
        lettre = choice([lettre,lettre.upper()])   #choisi entre la lettre et sa majuscule (les nombres restent comme ils sont)   
        sequence+=lettre
    return sequence

In [14]:
def genere_image(mot, size_x, size_y, posX,posY):
    img = Image.new(mode="1",size=(size_x,size_y),color=1)#voir apres pour size
    draw = ImageDraw.Draw(img)
    fnt = ImageFont.truetype(r"C:\Users\dnybs\trophee NSI\nouvelle direction\captcha\imageGeneration\fonts\Bongkerspop.ttf",size = 250)#voir apres pour size (en fct des données)
    draw.text((posX,posY),text = mot,fill = 0, font= fnt, spacing = 5)#pour que la variation sois pas tout le temps la meme
    return img

    un pixel a des coordonées (x,y) et on doit les passer dans une fonction qui va nous donner des coordonnes(u,v), avec la meme couleur
    on peut faire un truc du genre fonction sinus pour la coordonné x
    ont peut faire un truc du genre fct sinus pour la coordonné y 
    le pixel ou ils étaient est remplace par un pixel blanc

    ca change souvent car les pixels ont de grandes valeur dcp sur l'image 10 pixels c pas grand chose dcp ca passe vite en negatif
    dcp leffet est pas etalé mais concentre a certains endroits et absent dans dautre
    il faut peut etre normaliser les pixels entre 0 et 30 par exemple 

    il reste des choses a regler : 
    -normalisation (mettre toutes les valeurs dans le meme ordre de grandeur, pour que sin() varie doucement)
    y augmente linéairement 
    x augmente linéairement, deviens 0, puis re augmente linéairement -> en boucle

In [15]:
#ce filter a les amelioration du 2 + on distord sur les deux axes mais ducoup il faut faire ca l'un apres l'autre !
def filter(image,size_x,size_y):#ca sera juste distortion imo
#on veut distordre d'abord sur les x puis les y(on sen fout de lordre mais faut faire l'un apres lautre)
    for i in range(2):
        pixels_a_modifier = [] 
        for y in range(size_y):
            for x in range(size_x):
                if image.getpixel((x,y)) == 0:
                    pixels_a_modifier.append([x,y])
        for pixel_coor in pixels_a_modifier: #remplace par un pixel blanc mtn, pour ne pas supprimer ceux qu'on va mettre et qui sont la
            image.putpixel((pixel_coor[0],pixel_coor[1]),1)
        if i ==0: #distordre sur les y
            for p in pixels_a_modifier:
                x,y = p[0],p[1] 
                u = x 
                v = y-sin(x/100)*35 #corespond a une fct : v = f(x) = y+sin(x) SIN(X) PEUT ETRE NEGATIF
                #on peut tjrs changer le coeff de sin(x), le coeef de x
                u,v = ceil(u),ceil(v)
                if u <= size_x and v <= size_y:
                    image.putpixel((u,v),0)

        if i ==1: #distordre sur les x
            for p in pixels_a_modifier:
                x,y = p[0],p[1] 
                u = x +sin(y/50)*20
                v = y #corespond a une fct : v = f(x) = y+sin(x) SIN(X) PEUT ETRE NEGATIF
                #on peut tjrs changer le coeff de sin(y), le coeef de y
                u,v = ceil(u),ceil(v)
                if u <= size_x and v <= size_y:
                    image.putpixel((u,v),0)

In [16]:
def resizeImage(img,bord):
    largeur, hauteur = img.size
    #il faut couper sur les 4 cotes (en hauteur en haut et en bas, en largeur a droite et a gauche)
    #pour obtenir un rectangle : (largeur gauche,hauteur haut,largeur droite,hauteur bas)
    #en hauteur :
    #en bas : on sait a peu pres 
    for i in reversed(range(hauteur)):#on part du bas
        for j in range(largeur):#de gauche a droite
            if img.getpixel((j,i)) == 0:
                hauteurBas = i +bord
                break
        else:
            continue
        break                    
    #en haut : on sait a peu pres
    for i in range(hauteur):#on part du haut
        for j in range(largeur):#de gauche a droite
            if img.getpixel((j,i)) == 0:
                hauteurHaut = i-bord
                break
        else:
            continue
        break                    
    #en largeur:

    #a gauche : on sait a peu pres
    for j in range(largeur):#on part de la gauche
        for i in range(hauteur):#de haut en bas
            if img.getpixel((j,i)) == 0:
                largeurGauche = j-bord
                break
        else:
            continue
        break

    #a droite :

    for j in reversed(range(largeur)):# on part de la droite
        for i in range(hauteur):#de haut en bas
            if img.getpixel((j,i)) == 0:
                largeurDroite = j+bord
                break
        else:
            continue
        break
    img = img.resize(size = (largeurDroite,hauteurBas),box =(largeurGauche,hauteurHaut,largeurDroite,hauteurBas))
    return img

on peut optimiser : que on cherche les coins que a partir, de l'endroit (a peu pres) ou yaura le texte ET pour chercher les pixels



In [17]:
#on met le noise par dessus l'autre sinon le resize va etre bzr
#noise(img,img.size[0],img.size[1])
def noise(image,size_x,size_y,nbPoint):
    randomPose = [[(randint(-250,size_x), randint(-250,size_y)) for i in range(nbPoint)] for j in range(0,size_y,80)] #un tableau de toutes les positions tous les 80 pixels ver le bas et aleatoire vers sur x et y
    draw = ImageDraw.Draw(image)
    fnt = ImageFont.truetype(r"C:\Users\dnybs\trophee NSI\nouvelle direction\captcha\imageGeneration\fonts\Bongkerspop.ttf",size = 250)

    for j in randomPose:
        for coor in j:
            draw.text(coor,text = '.',fill = 0, font= fnt, spacing = 5)  

    return image


In [30]:

def genere_image_random():
    size_x = 2100 #on pourra passer ca en parametres
    size_y = 700
    posX =randint(50,200) #position du pixel le plus proche de la gauche
    posY =randint(60,300) #jsp pas trop
    for i in range(1):
        mot = create_sequence_random()
        print(mot)
        img = genere_image(mot, size_x,size_y,posX,posY)
        img = resizeImage(img,60)
        size_x,size_y = img.size[0],img.size[1]
        filter(img,size_x,size_y)  
        img = noise(img,size_x,size_y,5) 
        img.show()
        #img.save('C:\\Users\\dnybs\\trophee NSI\\preprocessImage\\image\\'+mot+'.png','PNG')
genere_image_random()

FUP6N8Bi5


In [2]:
choses = ["Truc","kekc"]
print(choses.index("Truc"))

0
